In [1]:
!brew services start typesense-server@0.24.0

Service `typesense-server@0.24.0` already started, use `brew services restart typesense-server@0.24.0` to restart.


In [2]:
!curl http://localhost:8108/health

{"ok":true}

In [3]:
import pandas as pd
import typesense

In [4]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost', # For Typesense Cloud use xxx.a1.typesense.net
    'port': '8108',      # For Typesense Cloud use 443
    'protocol': 'http'   # For Typesense Cloud use https
  }],
  'api_key': 'xyz',
  'connection_timeout_seconds': 2
})

In [5]:
books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string' },
    {'name': 'authors', 'type': 'string[]', 'facet': True },

    {'name': 'publication_year', 'type': 'int32', 'facet': True },
    {'name': 'ratings_count', 'type': 'int32' },
    {'name': 'average_rating', 'type': 'float' }
  ],
  'default_sorting_field': 'ratings_count'
}

client.collections.create(books_schema)

{'created_at': 1677855152,
 'default_sorting_field': 'ratings_count',
 'enable_nested_fields': False,
 'fields': [{'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'title',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': True,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'authors',
   'optional': False,
   'sort': False,
   'type': 'string[]'},
  {'facet': True,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'publication_year',
   'optional': False,
   'sort': True,
   'type': 'int32'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'ratings_count',
   'optional': False,
   'sort': True,
   'type': 'int32'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'average_rating',
   'optional': False,
   'sort': True,
   'type': 'float'}],
 'name': 'books',
 'num_documents': 0,
 'symbols_to_index': [],
 'token_separators': []}

In [6]:
with open('./tmp/books.jsonl') as jsonl_file:
    client.collections['books'].documents.import_(jsonl_file.read().encode('utf-8'))

In [7]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'sort_by'   : 'ratings_count:desc'
}

client.collections['books'].documents.search(search_parameters)


{'facet_counts': [],
 'found': 0,
 'hits': [],
 'out_of': 0,
 'page': 1,
 'request_params': {'collection_name': 'books',
  'per_page': 10,
  'q': 'harry potter'},
 'search_cutoff': False,
 'search_time_ms': 4}

In [10]:
!head ./tmp/books.jsonl

{"title":"The Hunger Games","authors":["Suzanne Collins"],"publication_year":2008,"id":"1","average_rating":4.34,"image_url":"https://images.gr-assets.com/books/1447303603m/2767052.jpg","ratings_count":4780653}
{"title":"Harry Potter and the Philosopher's Stone","authors":["J.K. Rowling"," Mary GrandPré"],"publication_year":1997,"id":"2","average_rating":4.44,"image_url":"https://images.gr-assets.com/books/1474154022m/3.jpg","ratings_count":4602479}
{"title":"Twilight","authors":["Stephenie Meyer"],"publication_year":2005,"id":"3","average_rating":3.57,"image_url":"https://images.gr-assets.com/books/1361039443m/41865.jpg","ratings_count":3866839}
{"title":"To Kill a Mockingbird","authors":["Harper Lee"],"publication_year":1960,"id":"4","average_rating":4.25,"image_url":"https://images.gr-assets.com/books/1361975680m/2657.jpg","ratings_count":3198671}
{"title":"The Great Gatsby","authors":["F. Scott Fitzgerald"],"publication_year":1925,"id":"5","average_rating":3.89,"image_url":"https:/